In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

SN_FILTER = True
WINDOW = 4
MAX_SEQ = 457
d_model = 256
num_layers = 12
num_heads = 8
dff = 256 * 2
emb_rate = 0.4
enc_rate = 0.4
vocab_size = 425
encoded_padded_value = 0

class positional_encoding_layer(tf.keras.layers.Layer):

    def __init__(self, vocab_size, maxlen, d_model):
        super().__init__()
        self.d_model = d_model
        self.pos_emb = self.positional_encoding(maxlen, d_model)
        self.supports_masking = True

    def call(self, x):
        maxlen = tf.shape(x)[-2]
        x = tf.math.multiply(x, tf.math.sqrt(tf.cast(self.d_model, tf.float32)))
        return x + self.pos_emb[0, :maxlen, :]

    def positional_encoding(self, maxlen, d_model):

        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
            return pos * angle_rates

        angle_rads = get_angles(np.arange(maxlen)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        pos_encoding = tf.cast(pos_encoding, dtype=tf.float32)
        
        return pos_encoding

def create_encoder():
    inp = tf.keras.layers.Input(shape=(None, ))
    
    x = tf.keras.layers.Embedding(input_dim=vocab_size + 1, output_dim=d_model, mask_zero=True)(inp)
    x = tf.keras.layers.Dropout(emb_rate)(inputs=x)
    x = positional_encoding_layer(vocab_size=vocab_size, maxlen=MAX_SEQ, d_model=d_model)(x)
    
    for i in range(num_layers):
        
        attn_output, _ = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(query=x, key=x, value=x, return_attention_scores=True)
        attn_output = tf.keras.layers.Dropout(enc_rate)(attn_output)
        mha_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        
        ffn_output = tf.keras.layers.Dense(dff, activation='relu')(mha_output)
        ffn_output = tf.keras.layers.Dense(d_model, activation='relu')(ffn_output)
        ffn_output = tf.keras.layers.Dropout(enc_rate)(ffn_output)
        
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(mha_output + ffn_output)
        
    x = tf.keras.layers.Dense(d_model / 2, activation='relu')(x)
    x = tf.keras.layers.Dense(d_model / 4, activation='relu')(x)
    x = tf.keras.layers.Dense(d_model / 8, activation='relu')(x)
    out = tf.keras.layers.Dense(2, activation='relu')(x)
       
    model = tf.keras.Model(inputs=inp, outputs=out)
    return model

encoder_model = create_encoder()

weights_path = 'model_weights.h5'
encoder_model.load_weights(weights_path)

In [2]:
with open('subset_dataset_val.pkl', 'rb') as file:
    val_data = pd.read_pickle(file)

In [3]:
def masked_mae_loss(react_true, react_pred):
    mask = tf.math.is_finite(react_true)
    react_true_masked = tf.boolean_mask(react_true, mask)
    react_pred_masked = tf.boolean_mask(react_pred, mask)
    loss = tf.keras.losses.mean_absolute_error(react_true_masked, react_pred_masked)
    return tf.reduce_mean(loss)

for input_data, react_true in val_data: 
    react_pred = encoder_model.predict(input_data)
    loss = masked_mae_loss(react_true, react_pred)
    print("Loss for the batch:", loss.numpy())

64/64 [==============================] - 11s 139ms/step
Loss for the batch: 0.20739539
64/64 [==============================] - 9s 136ms/step
Loss for the batch: 0.20769311
64/64 [==============================] - 9s 137ms/step
Loss for the batch: 0.20890361
64/64 [==============================] - 9s 137ms/step
Loss for the batch: 0.20996273
64/64 [==============================] - 9s 136ms/step
Loss for the batch: 0.20755301
64/64 [==============================] - 9s 136ms/step
Loss for the batch: 0.20893237
64/64 [==============================] - 9s 136ms/step
Loss for the batch: 0.21049546
64/64 [==============================] - 9s 136ms/step
Loss for the batch: 0.20913363
64/64 [==============================] - 9s 135ms/step
Loss for the batch: 0.20763488
64/64 [==============================] - 9s 135ms/step
Loss for the batch: 0.2095863
64/64 [==============================] - 9s 135ms/step
Loss for the batch: 0.20821513
64/64 [==============================] - 9s 136ms/step